In [1]:
import pandas as pd
import numpy as np

PATH = '/home/kai/data/kaggle/homecredit/'

train_ori = pd.read_pickle(PATH+'train_0.pkl')
test_ori = pd.read_pickle(PATH+'test_0.pkl')
prediction = pd.read_csv(PATH+'inter/06_21_total_1.csv')



# Cut for half

In [2]:
pred = prediction.copy()
train_df, test_df = train_ori.copy(), test_ori.copy()
print('done')
print(train_df.shape,test_df.shape)

done
(307511, 1754) (48744, 1753)


In [3]:
half = pred[pred['TARGET'] > 0.5]
index_half = half.index
print(len(half))

217


In [4]:
test_half = test_df.iloc[index_half]
test_half['TARGET'] = 1

# drop index_half for test_df, add train
test_df.drop(index_half, inplace = True)
train_df = pd.concat([train_df,test_half])
print(test_df.shape,train_df.shape)
# test.to_pickle(PATH+ 'inter/621_half_test_0')
# train.to_pickle(PATH+ 'inter/621_half_train_0')

/home/kai/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


(48527, 1753) (307728, 1754)


# Cut for small

In [5]:
pred = prediction.copy()
train, test = train_ori.copy(), test_ori.copy()
print(train.shape, test.shape)

small = pred[pred['TARGET']< 0.04]
index_small = small.index
print(index_small.sum())
test_small = test.iloc[index_half]
test_small['TARGET'] = 0

# drop index_half for test_df, add train
test.drop(index_small, inplace = True)
train = pd.concat([train,test_small])

test.to_pickle(PATH+ 'inter/621_small_test_0')
train.to_pickle(PATH+ 'inter/621_small_train_0')

NameError: name 'train_df' is not defined

In [ ]:
# Forked from excellent kernel : https://www.kaggle.com/jsaguiar/updated-0-792-lb-lightgbm-with-simple-features
# From Kaggler : https://www.kaggle.com/jsaguiar
# Just added a few features so I thought I had to make release it as well...

import numpy as np
import pandas as pd
import gc
import time
from contextlib import contextmanager
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import KFold, StratifiedKFold
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
PATH = '/home/kai/data/kaggle/homecredit/'

# lightgbm

In [5]:

import numpy as np
import pandas as pd
import gc
import time
from contextlib import contextmanager
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import KFold, StratifiedKFold
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
PATH = '/home/kai/data/kaggle/homecredit/'

In [ ]:
gc.collect()
# Cross validation model
stratified = False
debug = False

if stratified:
    folds = StratifiedKFold(n_splits= 5, shuffle=True, random_state=1001)
else:
    folds = KFold(n_splits= 5, shuffle=True, random_state=45)
# Create arrays and dataframes to store results
oof_preds = np.zeros(train_df.shape[0])
sub_preds = np.zeros(test_df.shape[0])
feature_importance_df = pd.DataFrame()
feats = [f for f in train_df.columns if f not in ['TARGET','SK_ID_CURR','SK_ID_BUREAU','SK_ID_PREV','index']]

for n_fold, (train_idx, valid_idx) in enumerate(folds.split(train_df[feats], train_df['TARGET'])):
    train_x, train_y = train_df[feats].iloc[train_idx], train_df['TARGET'].iloc[train_idx]
    valid_x, valid_y = train_df[feats].iloc[valid_idx], train_df['TARGET'].iloc[valid_idx]

    # LightGBM parameters found by Bayesian optimization
    clf = LGBMClassifier(
        nthread=16,
        n_estimators=10000,
        learning_rate=0.02,
        num_leaves=34,
        colsample_bytree=0.9497036,
        subsample=0.8715623,
        max_depth=8,
        reg_alpha=0.041545473,
        reg_lambda=0.0735294,
        min_split_gain=0.0222415,
        min_child_weight=39.3259775,
        silent=-1,
        verbose=-1, )

    clf.fit(train_x, train_y, eval_set=[(train_x, train_y), (valid_x, valid_y)], 
        eval_metric= 'auc', verbose= 100, early_stopping_rounds= 200)

    oof_preds[valid_idx] = clf.predict_proba(valid_x, num_iteration=clf.best_iteration_)[:, 1]
    sub_preds += clf.predict_proba(test_df[feats], num_iteration=clf.best_iteration_)[:, 1] / folds.n_splits

#     fold_importance_df = pd.DataFrame()
#     fold_importance_df["feature"] = feats
# #     fold_importance_df["importance"] = clf.feature_importances_
#     fold_importance_df["fold"] = n_fold + 1
#     feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    print('Fold %2d AUC : %.6f' % (n_fold + 1, roc_auc_score(valid_y, oof_preds[valid_idx])))
    del clf, train_x, train_y, valid_x, valid_y
    gc.collect()

print('Full AUC score %.6f' % roc_auc_score(train_df['TARGET'], oof_preds))

Training until validation scores don't improve for 200 rounds.
[100]	training's auc: 0.762957	valid_1's auc: 0.751565
[200]	training's auc: 0.790268	valid_1's auc: 0.772249
[300]	training's auc: 0.810084	valid_1's auc: 0.784228
[400]	training's auc: 0.822984	valid_1's auc: 0.789521
[500]	training's auc: 0.833518	valid_1's auc: 0.792822
[600]	training's auc: 0.8423	valid_1's auc: 0.794643
[700]	training's auc: 0.849896	valid_1's auc: 0.795923
[800]	training's auc: 0.857108	valid_1's auc: 0.796716
[900]	training's auc: 0.863513	valid_1's auc: 0.797245
[1000]	training's auc: 0.869462	valid_1's auc: 0.797633
[1100]	training's auc: 0.87527	valid_1's auc: 0.797952
[1200]	training's auc: 0.880557	valid_1's auc: 0.798356
[1300]	training's auc: 0.885666	valid_1's auc: 0.798388
[1400]	training's auc: 0.890339	valid_1's auc: 0.798442
[1500]	training's auc: 0.895094	valid_1's auc: 0.798525
[1600]	training's auc: 0.899653	valid_1's auc: 0.798676
[1700]	training's auc: 0.90402	valid_1's auc: 0.79859

In [ ]:
#     plt.title('LightGBM Features (avg over folds)')
#     plt.tight_layout()
#     plt.savefig('lgbm_importances01.png')
debug = False    
if not debug:
    print('a')
    test_df['TARGET'] = sub_preds
    test_df[['SK_ID_CURR', 'TARGET']].to_csv(PATH+'inter/0621_pred_half_0.csv', index= False)
# display_importances(feature_importance_df)

df =test_df[['SK_ID_CURR', 'TARGET']] 
print(len(df[df['TARGET']<0]),len(df[df['TARGET']>1/2]))

df

In [ ]:
test_half = pd.concat([test_half,test_df])

In [ ]:
tmp = test_ori[['SK_ID_CURR']]
tmp = tmp.merge(test_half, on = 'SK_ID_CURR', how = 'left')

In [ ]:
tmp.head()